In [1]:
import os
import json
import numpy as np
import pdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, mapping

In [2]:
def read_json(json_path):
    try:
        with open(json_path) as js:
            json_obj = json.load(js)
        return json_obj

    except FileNotFoundError:
        print('File not found.')

In [3]:
a = read_json("../pipeline.json")

In [4]:
a

{'pipeline': [{'polygon': '',
   'bounds': '',
   'filename': '',
   'type': 'readers.ept',
   'tag': 'readdata'},
  {'limits': 'Classification![7:7]',
   'type': 'filters.range',
   'tag': 'nonoise'},
  {'in_srs': 'EPSG:3857',
   'out_srs': 'EPSG:4326',
   'tag': 'reprojectUTM',
   'type': 'filters.reprojection'},
  {'filename': '../data/iowa.csv',
   'tag': 'writerslas',
   'type': 'writers.text'}]}

In [5]:
pipe=pdal.Pipeline(json.dumps(a))

In [6]:
MINX, MINY, MAXX, MAXY = [-93.759055, 41.925015, -93.766155, 41.935015]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

In [35]:
class UsgsLidar:
    
    def __init__(self, path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/", pipeline_json_path: str="pipeline.json") -> None:
            
    
        self.path = path
        self.input_epsg = 3857
    
        
    def fetch_polygon_boundaries(self, polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=4326, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    
    def read_csv(self, csv_path, missing_values=["n/a", "na", "undefined"]):
        try:
            df = pd.read_csv(csv_path, na_values=missing_values)
            return df

        except FileNotFoundError:
            print('File not found.')
            
    def fetch_pipeline (self, region: str, polygon: Polygon):
        url = f"{self.path}{region}/ept.json"
        boundary, poly = self.fetch_polygon_boundaries(polygon)
        
        a['pipeline'][0]['filename']= f"{self.path}{region}/ept.json"
        a['pipeline'][0]['polygon'] = poly
        a['pipeline'][0]['bounds'] = boundary
        pipeline = pdal.Pipeline(json.dumps(a))
        
        return pipeline
            


In [24]:
US = UsgsLidar()
df = US.read_csv("../data/iowa.csv")
shape, poly = US.fetch_polygon_boundaries(polygon)
print(poly)

POLYGON((-10437210.259858532 5149753.664381643, -10437210.259858532 5151249.971344454, -10438000.628243161 5151249.971344454, -10438000.628243161 5149753.664381643, -10437210.259858532 5149753.664381643))


In [40]:
region = "IA_FullState"
pipeline = US.fetch_pipeline(region, polygon)

In [43]:
pipe2 = pipeline.execute()

In [47]:
def create_geo_df(pipe, epsg):
    try:
        cloud_points = []
        elevations =[]
        geometry_points=[]
        for row in pipe.arrays[0]:
            lst = row.tolist()[-3:]
            cloud_points.append(lst)
            elevations.append(lst[2])
            point = Point(lst[0], lst[1])
            geometry_points.append(point)
        geodf = gpd.GeoDataFrame(columns=["elevation", "geometry"])
        geodf['elevation'] = elevations
        geodf['geometry'] = geometry_points
        geodf = geodf.set_geometry("geometry")
        geodf.set_crs(epsg = epsg, inplace=True)
        return geodf
    except RuntimeError as e:
        print(e)

In [48]:
create_geo_df(pipeline, 4326)

,elevation,geometry
0,314.54,POINT (-93.75906 41.92816)
1,314.55,POINT (-93.75906 41.92817)
2,314.69,POINT (-93.75909 41.92816)
3,314.70,POINT (-93.75910 41.92816)
4,314.97,POINT (-93.75911 41.92816)
...,...,...
913969,311.15,POINT (-93.76328 41.93497)
913970,307.90,POINT (-93.76490 41.93499)
913971,327.76,POINT (-93.76568 41.93491)
913972,315.14,POINT (-93.76131 41.92578)
